In [1]:
import sys
sys.path.append("..")
from pyzx.graph.graph import  Graph
from pyzx.linalg import Mat2
import numpy as np
debug=False # TODO remove the debug stat
sys.executable

'/home/aerylia/anaconda3/envs/QISKitenv/bin/python'

In [2]:
class Architecture():

    def __init__(self, graph=None, adjacency_matrix=None, backend=None):
        if graph is None:
            self.graph = Graph(backend=backend)
        else:
            self.graph = graph

        if adjacency_matrix is not None:
            # build the architecture graph
            n = adjacency_matrix.shape[0]
            self.vertices = self.graph.add_vertices(n)
            edges = [(self.vertices[row], self.vertices[col]) for row in range(n) for col in range(n) if adjacency_matrix[row, col] == 1]
            self.graph.add_edges(edges)
        else:
            self.vertices = [v for v in self.graph.vertices()]
        self.pre_calc_distances()
        self.qubit_map = [i for i,v in enumerate(self.vertices)]

    def pre_calc_distances(self):
        # https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm
        self.distances = {"upper": [self.FloydWarshall(until, upper=True) for until, v in enumerate(self.vertices)],
                          "full": [self.FloydWarshall(until, upper=False) for until, v in enumerate(self.vertices)]}

    def FloydWarshall(self, exclude_excl, upper=True):
        distances = {}
        vertices = self.vertices[exclude_excl:] if upper else self.vertices[:exclude_excl + 1]
        for edge in self.graph.edges():
            src, tgt = self.graph.edge_st(edge)
            if src in vertices and tgt in vertices:
                if upper:
                    distances[(src, tgt)] = (1, [(src, tgt)])
                    distances[(tgt, src)] = (1, [(tgt, src)])
                elif src > tgt:
                    distances[(src, tgt)] = (1, [(src, tgt)])
                else:
                    distances[(tgt, src)] = (1, [(tgt, src)])
        for v in vertices:
            distances[(v, v)] = (0, [])
        for i, v0 in enumerate(vertices):
            for j, v1 in enumerate(vertices if upper else vertices[:i+1]):
                for v2 in vertices if upper else vertices[: i+j+1]:
                    if (v0, v1) in distances.keys():
                        if (v1, v2) in distances.keys():
                            if (v0, v2) not in distances.keys() or distances[(v0,v2)][0] > distances[(v0, v1)][0] + distances[(v1, v2)][0]:
                                distances[(v0, v2)] = (distances[(v0, v1)][0] + distances[(v1, v2)][0], distances[(v0, v1)][1] + distances[(v1, v2)][1])
        return distances

    def steiner_tree(self, start, nodes, upper=True):
        # Approximated by calculating the all-pairs shortest paths and then solving the mininum spanning tree over the subset of vertices and their respective shortest paths.
        # https://en.wikipedia.org/wiki/Steiner_tree_problem#Approximating_the_Steiner_tree

        # The all-pairs shortest paths are pre-calculated and the mimimum spanning tree is solved with Prim's algorithm
        # https://en.wikipedia.org/wiki/Prim%27s_algorithm

        # returns an iterator that walks the steiner tree, yielding (adj_node, leaf) pairs. If the walk is finished, it yields None
        root = start
        # TODO deal with qubit mapping
        #root = self.qubit_map.index(start)
        #nodes = [self.qubit_map.index(node) for node in nodes]
        vertices = [root]
        edges = []
        #mode = "forward" if upper else "backward"
        debug and print(root, upper, nodes)
        distances = self.distances["upper"][root] if upper else self.distances["full"][root]
        while nodes != []:
            options = [(node, v, *distances[(v, node)]) for node in nodes for v in vertices if (v, node) in distances.keys()]
            best_option = min(options, key=lambda x: x[2])
            vertices.append(best_option[0])
            edges.extend(best_option[3])
            nodes.remove(best_option[0])
        edges = set(edges)
        steiner_pnts = list(set([i for edge in edges for i in edge if i not in vertices]))
        if debug:
            print("edges:", edges)
            print("nodes:", vertices)
            print("steiner points:", steiner_pnts)
        # First go through the tree to find and remove zeros
        vs = {root}
        while len(vs) > 0:
            es = [e for e in edges for v in vs if e[0] == v]
            old_vs = [v for v in vs]
            for edge in es:
                yield edge
                vs.add(edge[1])
            [vs.remove(v) for v in old_vs]
        yield None
        # Walk the tree bottom up to remove all ones.
        while len(edges) > 0:
            # find leaf nodes:
            debug and print(vertices, steiner_pnts, edges)
            vs_to_consider = [vertex for vertex in vertices if vertex not in [e0 for e0, e1 in edges]] + \
                             [vertex for vertex in steiner_pnts if vertex not in [e0 for e0,e1 in edges]]
            for v in vs_to_consider:
                # Get the edge that is connected to this leaf node
                for edge in [e for e in edges if e[1] == v]:
                    yield edge
                    edges.remove(edge)
                    #yield map(lambda i: self.qubit_map[i], edge)
        yield None
        
    def get_coupling_map(self, name):
        coupling_map = {v:[] for v in self.vertices}
        for v0, v1 in self.graph.edges():
            coupling_map[v0].append(v1)
            coupling_map[v1].append(v0)
        initial_layout = {(name, v): ('q', i) for i, v in enumerate(self.vertices)}
        return coupling_map, initial_layout

In [3]:
def gauss(matrix, architecture, full_reduce=False, x=None, y=None):
    global add_count
    add_count = 0
    def row_add(c0, c1):
        global add_count
        add_count += 1
        matrix.row_add(c0, c1)
        debug and print("Reducing", c0, c1)
        if x != None: x.row_add(c0, c1)
        if y != None: y.row_add(c1, c0)
    rows = matrix.rows()
    cols = matrix.cols()
    p_cols = []
    def steiner_reduce_upper(col, root, nodes):
        steiner_tree = architecture.steiner_tree(root, nodes, True)
        # Remove all zeros
        next_check = next(steiner_tree)
        debug and print("Step 1: remove zeros")
        zeros = []
        while next_check is not None:
            s0, s1 = next_check
            if matrix.data[s0, col] == 0:  # s1 is a new steiner point or root = 0
                zeros.append(next_check)
            next_check = next(steiner_tree)
        while zeros != []:
            s0, s1 = zeros.pop(-1)
            if matrix.data[s0, col] == 0:
                row_add(s1, s0)
                debug and print(matrix.data[s0, col], matrix.data[s1, col])
        # Reduce stuff
        debug and print("Step 2: remove ones")
        next_add = next(steiner_tree)
        while next_add is not None:
            s0, s1 = next_add
            row_add(s0, s1)
            debug and print("next")
            next_add = next(steiner_tree)
            debug and print(next_add)
        debug and print("Step 3: profit")

    def steiner_reduce_full(col, root, nodes):
        steiner_tree = architecture.steiner_tree(root, nodes, False)
        # Remove all zeros
        next_check = next(steiner_tree)
        debug and print("deal with zero root")
        if next_check is not None and matrix.data[next_check[0], col] == 0: #root is zero
            print("WARNING : Root is 0 => reducing non-pivot column", matrix.data)
        debug and print("Step 1: remove zeros", matrix.data[:,c])
        while next_check is not None:
            s0, s1 = next_check
            if matrix.data[s1, col] == 0:  # s1 is a new steiner point
                row_add(s0, s1)
            next_check = next(steiner_tree)
        # Reduce stuff
        debug and print("Step 2: remove ones", matrix.data[:,c])
        next_add = next(steiner_tree)
        while next_add is not None:
            s0, s1 = next_add
            row_add(s0, s1)
            next_add = next(steiner_tree)
        debug and print("Step 3: profit", matrix.data[:,c])
    pivot = 0
    for c in range(cols):
        nodes = [r for r in range(pivot, rows) if pivot==r or matrix.data[r][c] == 1]
        steiner_reduce_upper(c, pivot, nodes)
        if matrix.data[pivot][c] == 1:
            p_cols.append(c)
            pivot += 1
    print("Upper triangle form", matrix.data)
    rank = pivot
    debug and print(p_cols)
    if full_reduce:
        pivot -= 1
        for c in reversed(p_cols):
            debug and print(pivot, matrix.data[:,c])
            nodes = {r for r in range(0, pivot+1) if r==pivot or matrix.data[r][c] == 1}
            if len(nodes) > 1:
                steiner_reduce_full(c, pivot, list(nodes))
            pivot -= 1
            #[steiner_reduce_full(*n) for n in nodes]
            #steiner_reduce(c, nodes, False)
    print("ADD COUNT:", add_count)
    return rank

# Create the architecture

In [4]:
coupling_map = np.array([
        [0,1,0,0,0,1,0,0,0],
        [1,0,1,0,1,0,0,0,0],
        [0,1,0,1,0,0,0,0,0],
        [0,0,1,0,1,0,0,0,1],
        [0,1,0,1,0,1,0,1,0],
        [1,0,0,0,1,0,1,0,0],
        [0,0,0,0,0,1,0,1,0],
        [0,0,0,0,0,0,1,0,1],
        [0,0,0,1,0,0,0,1,0]
    ])
arch = Architecture(adjacency_matrix=coupling_map)

Test/visualise the creation of the architecture and precalculation of the all_pair_distances

In [5]:
print("Distances")
for mode in ["upper", "full"]:
    for i, dist_dict in enumerate(arch.distances[mode]):
        print(mode, i)
        for k,v in dist_dict.items():
            print(k, ":", v[0], v[1])
print("Steiner tree")
gen = arch.steiner_tree(1, [2,6])
print("top -> bottom")
next_node = next(gen)
while next_node is not None:
    print(next_node)
    v1, v2 = next_node
    next_node = next(gen)
# second run
print("bottom -> top")
next_node = next(gen)
while next_node is not None:
    print(next_node)
    v1, v2 = next_node
    next_node = next(gen)

Distances
upper 0
(0, 1) : 1 [(0, 1)]
(1, 0) : 1 [(1, 0)]
(0, 5) : 1 [(0, 5)]
(5, 0) : 1 [(5, 0)]
(1, 2) : 1 [(1, 2)]
(2, 1) : 1 [(2, 1)]
(1, 4) : 1 [(1, 4)]
(4, 1) : 1 [(4, 1)]
(2, 3) : 1 [(2, 3)]
(3, 2) : 1 [(3, 2)]
(3, 4) : 1 [(3, 4)]
(4, 3) : 1 [(4, 3)]
(3, 8) : 1 [(3, 8)]
(8, 3) : 1 [(8, 3)]
(4, 5) : 1 [(4, 5)]
(5, 4) : 1 [(5, 4)]
(4, 7) : 1 [(4, 7)]
(7, 4) : 1 [(7, 4)]
(5, 6) : 1 [(5, 6)]
(6, 5) : 1 [(6, 5)]
(6, 7) : 1 [(6, 7)]
(7, 6) : 1 [(7, 6)]
(7, 8) : 1 [(7, 8)]
(8, 7) : 1 [(8, 7)]
(0, 0) : 0 []
(1, 1) : 0 []
(2, 2) : 0 []
(3, 3) : 0 []
(4, 4) : 0 []
(5, 5) : 0 []
(6, 6) : 0 []
(7, 7) : 0 []
(8, 8) : 0 []
(0, 2) : 2 [(0, 1), (1, 2)]
(0, 4) : 2 [(0, 1), (1, 4)]
(0, 3) : 3 [(0, 1), (1, 2), (2, 3)]
(0, 8) : 4 [(0, 1), (1, 2), (2, 3), (3, 8)]
(0, 7) : 3 [(0, 1), (1, 4), (4, 7)]
(0, 6) : 2 [(0, 5), (5, 6)]
(1, 3) : 2 [(1, 2), (2, 3)]
(1, 5) : 2 [(1, 0), (0, 5)]
(1, 6) : 3 [(1, 0), (0, 5), (5, 6)]
(1, 7) : 2 [(1, 4), (4, 7)]
(1, 8) : 3 [(1, 2), (2, 3), (3, 8)]
(2, 0) : 2 [(2, 1), 

(2, 0) : 2 [(2, 1), (1, 0)]
(3, 0) : 3 [(3, 2), (2, 1), (1, 0)]
(3, 1) : 2 [(3, 2), (2, 1)]
(4, 0) : 2 [(4, 1), (1, 0)]
(4, 2) : 2 [(4, 3), (3, 2)]
(5, 1) : 2 [(5, 4), (4, 1)]
(5, 2) : 3 [(5, 4), (4, 3), (3, 2)]
(5, 3) : 2 [(5, 4), (4, 3)]
(6, 0) : 2 [(6, 5), (5, 0)]
(6, 1) : 3 [(6, 5), (5, 4), (4, 1)]
(6, 2) : 4 [(6, 5), (5, 4), (4, 3), (3, 2)]
(6, 3) : 3 [(6, 5), (5, 4), (4, 3)]
(6, 4) : 2 [(6, 5), (5, 4)]
full 7
(1, 0) : 1 [(1, 0)]
(5, 0) : 1 [(5, 0)]
(2, 1) : 1 [(2, 1)]
(4, 1) : 1 [(4, 1)]
(3, 2) : 1 [(3, 2)]
(4, 3) : 1 [(4, 3)]
(5, 4) : 1 [(5, 4)]
(7, 4) : 1 [(7, 4)]
(6, 5) : 1 [(6, 5)]
(7, 6) : 1 [(7, 6)]
(0, 0) : 0 []
(1, 1) : 0 []
(2, 2) : 0 []
(3, 3) : 0 []
(4, 4) : 0 []
(5, 5) : 0 []
(6, 6) : 0 []
(7, 7) : 0 []
(2, 0) : 2 [(2, 1), (1, 0)]
(3, 0) : 3 [(3, 2), (2, 1), (1, 0)]
(3, 1) : 2 [(3, 2), (2, 1)]
(4, 0) : 2 [(4, 1), (1, 0)]
(4, 2) : 2 [(4, 3), (3, 2)]
(5, 1) : 2 [(5, 4), (4, 1)]
(5, 2) : 3 [(5, 4), (4, 3), (3, 2)]
(5, 3) : 2 [(5, 4), (4, 3)]
(6, 0) : 2 [(6, 5), (5, 0)]
(

# Create a random matrix

In [8]:
size = coupling_map.shape
test_mat = None
inv = None
while test_mat is None:
    try:
        test_mat = np.random.randint(low=0, high=2, size=size)
        inv = np.linalg.inv(test_mat)
        u,s,vh = np.linalg.svd(test_mat, full_matrices=True)
    except:
        print("No inverse - try again")
        test_mat = None
test_mat

array([[0, 1, 0, 1, 0, 0, 0, 0, 1],
       [1, 1, 0, 1, 0, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 0, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 0, 1, 1, 1, 1],
       [0, 1, 0, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 0, 1, 1, 1, 1, 0],
       [1, 1, 0, 0, 1, 0, 1, 0, 0]])

## Original matrix decomposition 

In [9]:
original = Mat2(np.copy(test_mat))
rank = original.gauss(full_reduce=False)
print("Upper triangular form")
print(original.data)
rank = original.gauss(full_reduce=True)
print("Full reduction")
print(original.data)
print("Rank:", rank)

Upper triangular form
[[1 0 0 0 0 1 1 1 0]
 [0 1 0 1 0 0 0 0 1]
 [0 0 1 1 1 1 1 0 1]
 [0 0 0 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1]]
Full reduction
[[1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1]]
Rank: 9


In [10]:
original2 = Mat2(np.copy(test_mat))
print("Nr CNOTS:", len(original2.to_cnots(True)))

Nr CNOTS: 38


## Architectural matrix decomposition

In [11]:
matrix = Mat2(np.copy(test_mat))
rank = gauss(matrix, arch, full_reduce=True)
print("Full reduction")
print(matrix.data)
print("Rank:", rank)

Upper triangle form [[1 0 0 0 0 1 1 1 0]
 [0 1 0 1 0 0 0 0 1]
 [0 0 1 1 1 1 1 0 1]
 [0 0 0 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1]]
ADD COUNT: 50
Full reduction
[[1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1]]
Rank: 9


# Qiskit mapper to architecture 

In [12]:
from qiskit import QuantumProgram

ModuleNotFoundError: No module named '_a_star_mapping'

In [ ]:
import qiskit

In [60]:
class QiskitCircuit():
    
    def __init__(self, Q_SPECS):
        self.qp = QuantumProgram(specs=Q_SPECS)
        # get the circuit by Name
        self.circuit = self.qp.get_circuit('Circuit')

        # get the Quantum Register by Name
        self.quantum_r = self.qp.get_quantum_register('qr')

        # get the Classical Register by Name
        self.classical_r = self.qp.get_classical_register('cr')
    
    def row_add(self, q0, q1):
        self.circuit.cx(self.quantum_r[q0], self.quantum_r[q1])
        
    def compile(self, architecture, backend="local_qasm_simulator"):
        # Coupling map 
        qk_coupling_map, initial_layout=architecture.get_coupling_map("qr")
        qobj = circuit.qp.compile(["Circuit"], backend=backend, coupling_map=qk_coupling_map, initial_layout=initial_layout)
        return circuit.qp.get_compiled_qasm(qobj,'Circuit')
        

In [33]:
Q_SPECS = {
    'circuits': [{
        'name': 'Circuit',
        'quantum_registers': [{
            'name': 'qr',
            'size': 9
        }],
        'classical_registers': [{
            'name': 'cr',
            'size': 9
        }]}],
}

In [78]:
circuit = QiskitCircuit(Q_SPECS)
original = Mat2(np.copy(test_mat))
rank = original.gauss(x=circuit, full_reduce=True)
qasm_str = circuit.qp.get_qasm("Circuit")
print(qasm_str)
rank, len(qasm_str.split('\n'))-5

OPENQASM 2.0;
include "qelib1.inc";
qreg qr[9];
creg cr[9];
cx qr[0],qr[3];
cx qr[0],qr[4];
cx qr[0],qr[6];
cx qr[2],qr[1];
cx qr[1],qr[2];
cx qr[1],qr[4];
cx qr[1],qr[5];
cx qr[1],qr[8];
cx qr[2],qr[4];
cx qr[2],qr[6];
cx qr[2],qr[7];
cx qr[7],qr[3];
cx qr[3],qr[7];
cx qr[3],qr[8];
cx qr[4],qr[5];
cx qr[4],qr[6];
cx qr[7],qr[5];
cx qr[5],qr[7];
cx qr[7],qr[6];
cx qr[6],qr[7];
cx qr[7],qr[8];
cx qr[8],qr[5];
cx qr[6],qr[4];
cx qr[7],qr[0];
cx qr[8],qr[6];
cx qr[8],qr[7];
cx qr[7],qr[3];
cx qr[5],qr[1];
cx qr[5],qr[3];
cx qr[4],qr[0];
cx qr[3],qr[0];
cx qr[3],qr[1];



(9, 32)

In [79]:
qasm_str = circuit.compile(arch)
print(qasm_str)
len(qasm_str.split('\n'))-5

QasmError: 'Include file c:\\anaconda3\\envs\\pyzx_env\\lib\\site-packages\\qiskit\\qasm\\libs\\qelib1.inc cannot be found, line 2, file '

In [63]:
circuit2 = QiskitCircuit(Q_SPECS)
matrix = Mat2(np.copy(test_mat))
rank = gauss(matrix, arch, x=circuit2, full_reduce=True)
qasm_str = circuit2.qp.get_qasm("Circuit")
print(qasm_str)
rank, len(qasm_str.split('\n'))-5

Upper triangle form [[1 0 0 0 1 0 0 1 0]
 [0 1 1 1 0 1 1 0 1]
 [0 0 1 1 0 0 0 1 0]
 [0 0 0 1 1 1 1 1 0]
 [0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0]]
ADD COUNT: 47
OPENQASM 2.0;
include "qelib1.inc";
qreg qr[9];
creg cr[9];
cx qr[2],qr[1];
cx qr[6],qr[5];
cx qr[1],qr[2];
cx qr[5],qr[6];
cx qr[0],qr[1];
cx qr[0],qr[5];
cx qr[1],qr[4];
cx qr[4],qr[3];
cx qr[3],qr[4];
cx qr[2],qr[3];
cx qr[4],qr[5];
cx qr[4],qr[7];
cx qr[3],qr[4];
cx qr[5],qr[6];
cx qr[4],qr[5];
cx qr[6],qr[5];
cx qr[7],qr[8];
cx qr[6],qr[7];
cx qr[5],qr[6];
cx qr[7],qr[8];
cx qr[6],qr[7];
cx qr[8],qr[7];
cx qr[7],qr[8];
cx qr[7],qr[4];
cx qr[6],qr[5];
cx qr[4],qr[1];
cx qr[7],qr[6];
cx qr[7],qr[4];
cx qr[6],qr[5];
cx qr[5],qr[4];
cx qr[5],qr[0];
cx qr[3],qr[2];
cx qr[4],qr[3];
cx qr[5],qr[4];
cx qr[6],qr[5];
cx qr[5],qr[0];
cx qr[3],qr[2];
cx qr[4],qr[3];
cx qr[5],qr[4];
cx qr[1],qr[0];
cx qr[3],qr[2];
cx qr[4],qr[1];
cx qr[4],qr[3];
cx qr[3],qr[2];
cx qr[1],qr[

(8, 47)

In [56]:
# Coupling map 
qk_coupling_map = {0: [1, 2, 3, 4, 5, 6, 7, 8]}
# Place the qubits on a triangle in the bow-tie
initial_layout={("qr", i): ("q", i) for i in range(9)}
qobj = circuit.qp.compile(["Circuit"], backend="local_qasm_simulator", coupling_map=qk_coupling_map, initial_layout=initial_layout)
new_str = circuit.qp.get_compiled_qasm(qobj,'Circuit')
print(new_str)

OPENQASM 2.0;
include "qelib1.inc";
qreg q[9];
creg cr[9];
u2(0,3.14159265358979) q[3];
u2(0,3.14159265358979) q[2];
u2(0,3.14159265358979) q[0];
cx q[0],q[2];
u2(0,3.14159265358979) q[2];
u2(0,3.14159265358979) q[0];
cx q[0],q[2];
cx q[0],q[1];
cx q[0],q[6];
u2(0,3.14159265358979) q[0];
u2(0,3.14159265358979) q[6];
cx q[0],q[6];
u2(0,3.14159265358979) q[6];
u2(0,3.14159265358979) q[0];
cx q[0],q[6];
u2(0,3.14159265358979) q[0];
u2(0,3.14159265358979) q[1];
u2(0,3.14159265358979) q[2];
cx q[0],q[2];
u2(0,3.14159265358979) q[2];
cx q[0],q[3];
u2(0,3.14159265358979) q[3];
u2(0,3.14159265358979) q[0];
cx q[0],q[6];
u2(0,3.14159265358979) q[0];
u2(0,3.14159265358979) q[6];
cx q[0],q[6];
u2(0,3.14159265358979) q[6];
u2(0,3.14159265358979) q[0];
cx q[0],q[6];
u2(0,3.14159265358979) q[0];
cx q[0],q[1];
u2(0,3.14159265358979) q[0];
cx q[0],q[3];
u2(0,3.14159265358979) q[0];
u2(0,3.14159265358979) q[3];
cx q[0],q[3];
u2(0,3.14159265358979) q[3];
u2(0,3.14159265358979) q[0];
cx q[0],q[3];
cx q[0

In [71]:
circuit2.qp